In [19]:
#! /usr/bin/env python

###################################################################################
# Copyright   2015, Pittsburgh Supercomputing Center (PSC).  All Rights Reserved. #
# =============================================================================== #
#                                                                                 #
# Permission to use, copy, and modify this software and its documentation without #
# fee for personal use within your organization is hereby granted, provided that  #
# the above copyright notice is preserved in all copies and that the copyright    #
# and this permission notice appear in supporting documentation.  All other       #
# restrictions and obligations are defined in the GNU Affero General Public       #
# License v3 (AGPL-3.0) located at http://www.gnu.org/licenses/agpl-3.0.html  A   #
# copy of the license is also provided in the top level of the source directory,  #
# in the file LICENSE.txt.                                                        #
#                                                                                 #
###################################################################################

import logging

import sys
import os.path
from phacsl.utils.collections.phacollections import SingletonMetaClass
import pyrheautils
from collections import deque, defaultdict
from facilitybase import CareTier
sys.path.append(os.path.join(os.path.dirname(pyrheautils.__file__), '../tools'))
from tools_util import readModelInputs, getFacDict
import yaml

runDesc = '/home/welling/git/pyRHEA_github/src/sim/week_run_OC.yaml'

inputDict = readModelInputs(runDesc)
facDict = getFacDict(inputDict)

# Drop annoying community entries
dropL = []
for abbrev, rec in facDict.items():
    if rec['category'] == 'COMMUNITY':
        dropL.append(abbrev)
for abbrev in dropL:
    del facDict[abbrev]

print facDict.keys()



['CSNC', 'HOAG', 'LPAL', 'GGMC', 'MANO', 'COVI', 'CVSB', 'SCGL', 'CVLH', 'PACI', 'BGSJ', 'SEAC', 'SJUD', 'ANHC', 'FLAG', 'FVRH', 'KNOT', 'SMMC', 'EVER', 'HGOR', 'WAMC', 'PALM', 'LAKE', 'COLL', 'GGCH', 'STAN', 'FPCC', 'GORD', 'MSLG', 'CHAP', 'FOCC', 'KAIA', 'HVAL', 'CARE', 'SCRT', 'SNMR', 'BEAC', 'VICT', 'PKVW', 'UCIM', 'VIVA', 'LHAB', 'HSOU', 'WAEC', 'TOWN', 'HELP', 'OCGB', 'EXTW', 'ROYL', 'LEIS', 'ACNC', 'ALTA', 'WINA', 'NSUB', 'COCT', 'FREE', 'PREG', 'ELIZ', 'PSUP', 'CVPL', 'TERR', 'PANA', 'MESA', 'LAMC', 'BPNC', 'WINF', 'PVIS', 'GREE', 'HVCC', 'ORRH', 'EDNA', 'ANRG', 'BVCC', 'LPIH', 'NNRC', 'BGYL', 'KINB', 'KINO', 'KINW', 'CAPO', 'ATCC', 'MISS', 'ANGL', 'ALAM', 'OCMM', 'WLNT', 'RGNT', 'QUAK', 'NEWO', 'GPCC', 'SJOS', 'PLAC', 'HBHO']


In [20]:
import phacsl.utils.formats.csv_tools as csv_tools
import phacsl.utils.formats.yaml_tools as yaml_tools
from pyrheautils import pathTranslate

snfDspFname = 'Hospital_and_NH_Disposition_Outcomes_for_RHEA_2.0_-_Adult_Only_-_UPDATED_-_11-15-2017_NH_Outcomes.csv'

snfPath = pathTranslate('$(MODELDIR)/%s' % snfDspFname)
with open(snfPath, 'rU') as f:
    snfKeys, snfRecs = csv_tools.parseCSV(f)
print snfKeys
snfCsvD = {r['NH']: r for r in snfRecs}

[u'NH', u'Total Annual Discharges', u'(1) Patient Died', u'(2) Directly transferred to NH', u'(3) Directly transferred to hospital', u'(4) Discharged to Home**']


In [21]:
hospDspFname = 'Hospital_and_NH_Disposition_Outcomes_for_RHEA_2.0_-_Adult_Only_-_UPDATED_-_11-15-2017_Hosp_Outcomes.csv'

hospPath = pathTranslate('$(MODELDIR)/%s' % hospDspFname)
with open(hospPath, 'rU') as f:
    hospKeys, hospRecs = csv_tools.parseCSV(f)
print hospKeys
hospCsvD = {r['HOSPITAL']: r for r in hospRecs}

[u'HOSPITAL', u'Total Annual Discharges', u'(1) Patient Died', u'(2) Directly transferred to NH', u'(3) Directly transferred to hospital', u'(4) Indirectly transferred to hospital within 365 days', u'(5) Discharged and NOT readmitted to hospital within 365 days']


In [22]:
snfChrFname = 'OC_Hospital_and_Nursing_Home_Characteristics_for_RHEA_2.0_-_Adult_Only_-_09-15-2017_FINAL_NH_CHAR.csv'

with open(pathTranslate('$(MODELDIR)/%s' % snfChrFname), 'rU') as f:
    snfChrKeys, snfChrRecs = csv_tools.parseCSV(f)
print snfChrKeys
snfChrD = {r['RHEA 2.0 CODE']: r for r in snfChrRecs}

[u'CURRENT NAME (2017)', u'RHEA 2.0 CODE', u'"ANNUAL ADMISSIONS ADULT ONLY"', u'TOTAL LICENSED BEDS', u'AVERAGE DAILY CENSUS', u'', u'AVERAGE DAILY CENSUS EXCEEDS TOTAL BEDS']


In [23]:
hospChrFname = 'OC_Hospital_and_Nursing_Home_Characteristics_for_RHEA_2.0_-_Adult_Only_-_09-15-2017_FINAL_HOSP_CHAR.csv'

with open(pathTranslate('$(MODELDIR)/%s' % hospChrFname), 'rU') as f:
    hospChrKeys, hospChrRecs = csv_tools.parseCSV(f)
print hospChrKeys
hospChrD = {r['RHEA 2.0 CODE']: r for r in hospChrRecs}

[u'CURRENT NAME (2017)', u'RHEA 2.0 CODE', u'"ANNUAL ADMISSIONS ADULT ONLY"', u'TOTAL ADULT BEDS', u'TOTAL ADULT ICU BEDS', u'TOTAL ADULT NON-ICU BEDS', u'AVERAGE DAILY CENSUS ADULT ONLY', u'', u'AVERAGE DAILY CENSUS EXCEEDS TOTAL BEDS']


In [24]:
hospLOSFname = 'OC_Hospital_MRSA-Stratified_LOS_Distributions_for_RHEA_2.0_-_Adult_Only_-_09-18-2017_FINAL_MRSAMINUS_LOS_ALL.csv'
with open(pathTranslate('$(MODELDIR)/%s' % hospLOSFname), 'rU') as f:
    hospLOSKeys, hospLOSRecs = csv_tools.parseCSV(f)
print hospLOSKeys
hospLOSD = {r['CODE']: r for r in hospLOSRecs}


[u'CODE', u'N', u'LN_MEAN', u'LN_STD_DEV', u'LN_MEDIAN', u'LN_RANGE', u'LN_IQR']


In [25]:
with open(pathTranslate('$(MODELDIR)/fac_to_cat_marginalized_direct.yaml'), 'rU') as f:
    facToCatD = yaml.load(f)
print facToCatD.keys()

['ACNC', 'ALAM', 'ALTA', 'ANGL', 'ANHC', 'ANRG', 'ATCC', 'BEAC', 'BGSJ', 'BGYL', 'BPNC', 'BVCC', 'CAPO', 'CARE', 'CHAP', 'COCT', 'COLL', 'COVI', 'CSNC', 'CVLH', 'CVPL', 'CVSB', 'EDNA', 'ELIZ', 'EVER', 'EXTW', 'FLAG', 'FOCC', 'FPCC', 'FREE', 'FVRH', 'GGCH', 'GGMC', 'GORD', 'GPCC', 'GREE', 'HBHO', 'HELP', 'HGOR', 'HOAG', 'HSOU', 'HVAL', 'HVCC', 'KAIA', 'KINB', 'KINO', 'KINW', 'KNOT', 'LAKE', 'LAMC', 'LEIS', 'LHAB', 'LPAL', 'LPIH', 'MANO', 'MESA', 'MISS', 'MSLG', 'NEWO', 'NNRC', 'NSUB', 'OCGB', 'OCMM', 'ORRH', 'PACI', 'PALM', 'PANA', 'PKVW', 'PLAC', 'PREG', 'PSUP', 'PVIS', 'QUAK', 'RGNT', 'ROYL', 'SCGL', 'SCRT', 'SEAC', 'SJOS', 'SJUD', 'SMMC', 'SNMR', 'STAN', 'TERR', 'TOWN', 'UCIM', 'VICT', 'VIVA', 'WAEC', 'WAMC', 'WINA', 'WINF', 'WLNT']


In [26]:
with open(pathTranslate('$(MODELDIR)/direct_transfer_counts.yaml'), 'rU') as f:
    directTransferCts = yaml.load(f)
totTransIn = defaultdict(lambda: 0.0)
for src, rec in directTransferCts.items():
    for dst, ct in rec.items():
        totTransIn[dst] += ct
print totTransIn['PSUP']

712.0


In [27]:
with open(pathTranslate('$(MODELDIR)/los_model_fit_parms_fixedmean.csv'), 'rU') as f:
    nhLOSKeys, nhLOSRecs = csv_tools.parseCSV(f)
print nhLOSKeys
nhLOSD = {r['abbrev']: r for r in nhLOSRecs}
print nhLOSD['PSUP']

[u'abbrev', u'k', u'mu', u'sigma', u'mu2', u'sigma2', u'lmda', u'lmda2', u'lnLikPerSample', u'nsamples', u'nbins', u'pdf', u'notes']
{u'nbins': 4372, u'lmda2': u'NA', u'mu2': u'NA', u'k': 0.9969637918471277, u'mu': 2.9148072085931216, u'abbrev': u'PSUP', u'lnLikPerSample': u'[-3.45874152]', u'lmda': 0.00023682633612327828, u'pdf': u'"$0*lognorm(mu=$1,sigma=$2)+(1-$0)*expon(lambda=$3)"', u'nsamples': 4372, u'sigma': 0.8964435061665147, u'notes': u'', u'sigma2': u'NA'}


In [28]:
with open(pathTranslate('$(MODELDIR)/los_model_fit_nh_two_weibull.csv'), 'rU') as f:
    wbNHLOSKeys, wbNHLOSRecs = csv_tools.parseCSV(f)
print wbNHLOSKeys
wbNHLOSD = {r['abbrev']: r for r in wbNHLOSRecs}
print wbNHLOSD['NEWO']

[u'abbrev', u'k', u'k_from_line_list', u'lnLikPerSample', u'notes', u'nsamples', u'scale1', u'scale2', u'shape1', u'shape2']
{u'scale1': 20.559685, u'scale2': 162.329318, u'notes': u'', u'shape1': 1.352236, u'shape2': 0.610925, u'abbrev': u'PSUP', u'lnLikPerSample': u'', u'k_from_line_list': 0.0445083836175515, u'nsamples': 4372, u'k': 0.9023011568387707}


In [ ]:
with open(pathTranslate('$(MODELDIR)/los_model_fit_nh_split_fates.csv'), 'rU') as f:
    sfNHLOSKeys, sfNHLOSRecs = csv_tools.parseCSV(f)
print sfNHLOSKeys
sfNHLOSD = {r['abbrev']: r for r in sfNHLOSRecs}
print sfNHLOSD['NEWO']

In [32]:
with open(pathTranslate('$(MODELDIR)/los_model_fit_nh_split_fates.csv'), 'rU') as f:
    sfNHLOSKeys, sfNHLOSRecs = csv_tools.parseCSV(f)
print sfNHLOSKeys
sfNHLOSD = {r['abbrev']: r for r in sfNHLOSRecs}
print sfNHLOSD['PSUP']

[u'abbrev', u'chisqr_per_sample', u'nsamples', u'parms0', u'parms1', u'parms2', u'parms3', u'pdf']
{u'chisqr_per_sample': 0.03329094967086047, u'abbrev': u'PSUP', u'pdf': u'"$0*lognorm(mu=$1,sigma=$2)+(1-$0)*expon(lambda=$3)"', u'nsamples': 4310, u'parms1': 2.6656521074716064, u'parms0': 0.7477, u'parms3': 0.00672763322871806, u'parms2': 0.721964558446551}


In [33]:
from scipy.stats.distributions import rv_continuous, lognorm
import math
newFacDict = {}
directMtxFname = 'OC_Direct_Transfer_Matrices_for_RHEA_2.0_-_Adult_Only_-_09-15-2017_FINAL.xlsx'
for abbrev, rec in facDict.items():
    newR = rec.copy()
    if rec['category'] == 'NURSINGHOME':
        transProv = ('(%s summed by dest category and normalized) * (%s total discharges * frac to NH or HOSP)'
                     % (directMtxFname, snfDspFname))
        meanPopProv = ("%s col 'AVERAGE DAILY CENSUS' or 98%% of 'TOTAL LICENSED BEDS', whichever is lower"
                       % snfChrFname)
        nBedsProv = "%s col 'TOTAL LICENSED BEDS'" % snfChrFname
        if abbrev in snfCsvD:
            csvR = snfCsvD[abbrev]
            totDsch = csvR['Total Annual Discharges']
            totTrans = totDsch * (1.0 - (csvR[u'(4) Discharged to Home**'] + csvR[u'(1) Patient Died']))
            
            tNHFrac = facToCatD[abbrev]['NURSINGHOME'] if 'NURSINGHOME' in facToCatD[abbrev] else 0.0
            tLTACFrac = facToCatD[abbrev]['LTAC'] if 'LTAC' in facToCatD[abbrev] else 0.0
            tHospFrac = facToCatD[abbrev]['HOSPITAL'] if 'HOSPITAL' in facToCatD[abbrev] else 0.0
            newR['totalDischarges'] = {'value': totDsch,
                                      'prov': '%s col B' % snfDspFname}
            
            newR['totalTransfersOut'] = [
                {'category': 'NURSINGHOME', 'count': {'value': totTrans*tNHFrac, 'prov': transProv}},
                {'category': 'LTAC', 'count': {'value': totTrans*tLTACFrac, 'prov': transProv}},
                {'category': 'HOSPITAL', 'count': {'value': totTrans*tHospFrac, 'prov': transProv}}
            ]
            newR['nBeds'] = {'value': snfChrD[abbrev]['TOTAL LICENSED BEDS'],
                            'prov': nBedsProv}
            meanPop = min(snfChrD[abbrev]['AVERAGE DAILY CENSUS'], 0.98 * newR['nBeds']['value'])
            newR['meanPop'] = {'value': meanPop,
                              'prov': meanPopProv}
            newR['meanLOSICU'] = {'value': 0.0, 'prov': 'NURSINGHOMEs do not have ICUs'}
            newR['fracAdultPatientDaysICU'] = {'value': 0.0, 'prov': 'NURSINGHOMEs do not have ICUs'}
            newR['totalTransfersIn'] = {'value': totTransIn[abbrev],
                                       'prov': ('translate_direct_transfer_data_OC.py d6d33794'
                                                + ' with input CSVs bdfa32e4')}
            #losR = nhLOSD[abbrev]
            #newR['losModel'] = {'negLogLikPerSample': -float(losR['lnLikPerSample'].strip('[]')),
            #                    'nsamp': losR['nsamples'],
            #                    'parms': [losR['k'], losR['mu'], losR['sigma'], losR['lmda']],
            #                    'pdf': losR['pdf'].strip('"'),
            #                    'prov': ('los_model_fit_fixedmean.py e308e809 applied to '
            #                             'OC_Nursing_Home_LOS_Line-Lists_for_RHEA_2.0_-_2011-2015_-_Adult_Only_-_09-29-2017_FINAL.xlsx')
            #                    }
            #losR = wbNHLOSD[abbrev]
            #newR['losModel'] = {'parms': [1.0 - losR['k'], losR['shape2'], losR['scale2'],
            #                              losR['shape1'], losR['scale1']],
            #                    'pdf': '$0*weibull(k=$1, lmda=$2)+(1-$0)*weibull(k=$3, lmda=$4)',
            #                    'prov': 'nh-fitting-2weibull.R and convert.py 213f4c53 based on OC_Nursing_Home_LOS_Line-Lists_for_RHEA_2.0_-_2011-2015_-_Adult_Only_-_09-29-2017_FINAL_NH_LOS_Line_List.csv'}
            losR = sfNHLOSD[abbrev]
            newR['losModel'] = {'chiSqrPerSample': losR['chisqr_per_sample'],
                                'parms': [losR['parms0'], losR['parms1'], losR['parms2'], losR['parms3']],
                                'pdf': losR['pdf'].strip('"').strip("'"),
                                'prov': 'OC2013_NH_LOS_model_split_fates.ipynb e68fb60b'}
            snfChrR = snfChrD[abbrev]
            newR['meanLOS'] = {'value': (365.*newR['meanPop']['value']
                                         / float(snfChrR['"ANNUAL ADMISSIONS ADULT ONLY"'])),
                               'prov': ('%s 365*min("AVERAGE DAILY CENSUS", .98*"TOTAL LICENSED BEDS")/"ANNUAL ADMISSIONS ADULT ONLY"'
                                        % snfChrFname)
                               }
        else:
            print 'No data for %s' % abbrev
    elif rec['category'] in ['HOSPITAL', 'LTAC']:
        transProv = ('(%s summed by dest category and normalized) * (%s total discharges * frac to NH or HOSP)'
                     % (directMtxFname, hospDspFname))
        meanPopProv = "%s col 'AVERAGE DAILY CENSUS'" % hospChrFname
        nBedsProv = "%s col 'TOTAL LICENSED BEDS'" % hospChrFname
        if abbrev in hospCsvD:
            csvR = hospCsvD[abbrev]
            totDsch = csvR['Total Annual Discharges']
            totTrans = totDsch * (1.0 - (csvR[u'(4) Indirectly transferred to hospital within 365 days']
                                         + csvR[u'(5) Discharged and NOT readmitted to hospital within 365 days']
                                         + csvR[u'(1) Patient Died']))
            tNHFrac = facToCatD[abbrev]['NURSINGHOME'] if 'NURSINGHOME' in facToCatD[abbrev] else 0.0
            tLTACFrac = facToCatD[abbrev]['LTAC'] if 'LTAC' in facToCatD[abbrev] else 0.0
            tHospFrac = facToCatD[abbrev]['HOSPITAL'] if 'HOSPITAL' in facToCatD[abbrev] else 0.0
            newR['totalDischarges'] = {'value': totDsch,
                                      'prov': '%s col B' % hospDspFname}
            newR['totalTransfersOut'] = [
                {'category': 'NURSINGHOME', 'count': {'value': totTrans*tNHFrac, 'prov': transProv}},
                {'category': 'LTAC', 'count': {'value': totTrans*tLTACFrac, 'prov': transProv}},
                {'category': 'HOSPITAL', 'count': {'value': totTrans*tHospFrac, 'prov': transProv}}
            ]
            newR['meanPop'] = {'value': hospChrD[abbrev][u'AVERAGE DAILY CENSUS ADULT ONLY'],
                            'prov': meanPopProv}
            newR['nBeds'] = {'value':hospChrD[abbrev]['TOTAL ADULT BEDS'],
                            'prov': nBedsProv}
            losMu = hospLOSD[abbrev]['LN_MEAN']
            losSigma = hospLOSD[abbrev]['LN_STD_DEV']
            losRV = lognorm(losSigma, scale=math.exp(losMu), loc=0.0)
            meanLOS = losRV.mean()
            newR['meanLOS'] = {'value': float(losRV.mean()), 
                               'prov':'%s mean of lognorm(sigma=LN_STD_DEV, mu=LN_MEAN)'% hospLOSFname}
            newR['losModel'] = {'parms': [losMu, losSigma], 'pdf': 'lognorm(mu=$0,sigma=$1)',
                               'prov': '%s lognorm(sigma=LN_STD_DEV, mu=LN_MEAN)' % hospLOSFname}
            newR['totalTransfersIn'] = {'value': totTransIn[abbrev],
                                       'prov': ('translate_direct_transfer_data_OC.py d6d33794'
                                                + ' with input CSVs bdfa32e4')}
            newR['scaleLengthOfStay'] = {'value': 1.0, 'prov': 'default is no scaling'}

    newFacDict[abbrev] = newR
yaml_tools.save_all(pathTranslate('$(MODELDIR)/facilityfactsUpdated'), newFacDict.values())

In [ ]:
from scipy.stats.distributions import rv_continuous, lognorm, weibull_min
import math
# 1.94 is the ln of the mean
# .83 is the ln of stdv
mu = 1.94
sigma = 0.83
rv = lognorm(sigma, math.exp(mu))
print rv.mean(), rv.median()


In [ ]:
mean = math.exp(1.94)
stdv = math.exp(0.83)
#mu = math.log(mean) - (0.5*stdv*stdv)
mu = mean - (0.5*stdv*stdv)
#rv = lognorm(stdv, scale=math.exp(mu), loc=0.0)
rv1 = lognorm(0.83, scale=mu, loc=0.0)
print rv1.median()

In [ ]:
mu = 1.94
#mu = 0.0
sigma = 0.83
#sigma = 1.0
rv2 = lognorm(sigma, scale=math.exp(mu), loc=0.0)
print 'mean:   ', rv2.mean()
print 'median: ', rv2.median()

In [ ]:
mu = 1.94
#mu = 0.0
sigma = 0.83
#sigma = 1.0
rv3 = lognorm(sigma, scale=math.exp(mu), loc=10.0)
print 'mean:   ', rv3.mean()
print 'median: ', rv3.median()

In [ ]:
k, shape1, scale1, shape2, scale2 = 0.5416364701, 0.6074742465, 165.0048283694, 1.3523950354, 20.4852828978

rv4 = weibull_min(shape1, scale=scale1)
rv5 = weibull_min(shape2, scale=scale2)
#mu = 0.0
sigma = 0.83
#sigma = 1.0
rv3 = lognorm(sigma, scale=math.exp(mu), loc=10.0)
print 'mean:   ', rv4.mean(), rv5.mean()
print 'median: ', rv4.median(), rv5.median()

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
axes = plt.subplot(111)
x = np.linspace(0, 31)
axes.plot(x, rv1.pdf(x), label='rv1')
axes.plot(x, rv2.pdf(x), label='rv2')
axes.plot(x, rv3.pdf(x), label='rv3')
axes.plot(x, rv5.pdf(x), label='rv5')
axes.legend()
plt.show()

In [ ]:
lnRV = lognorm(8.96443506e-01 , scale=math.exp(2.91480721e+00))
print lnRV.mean()

In [ ]:
from scipy.stats.distributions import expon
expRV = expon(scale=(1.0/2.36826336e-04))
print expRV.mean()